In [1]:
from transpile_benchy.main import Benchmark
from qiskit.transpiler import CouplingMap
from virtual_swap.pass_managers import SabreCNS, Baseline

coupling = CouplingMap.from_grid(6, 6)
t0 = SabreCNS(coupling)
t1 = Baseline(coupling)

b = Benchmark(t0, t1, size="small", prepath="../../../../transpile_benchy")
b.run()

# plot and save as svg
b.plot(save=True)

print(b.depth_list)

NotImplementedError: 